## InfluxDB 연결 설정
InfluxDB 연결 정보 설정 및 장비 맵핑

In [1]:
%pip install influxdb_client

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import pytz
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.stats import zscore
from datetime import datetime, timedelta
from datetime import datetime, timedelta
from influxdb_client import InfluxDBClient

In [4]:
# InfluxDB 설정 정보
url = "http://133.186.144.22:8086"
token = "r3Ecro-rJQ82UpyNScnHXYDZ3KaE45AzweCXz6QIv2jeo7eOP4hL4-A9uKvkAVQDg_xavWorGUGZn7MI_sPCwg=="
org = "smoothing"

# Device ID와 위치를 매핑
location_mapping = {
    '24e124126d152919': 'indoor',
    '24e124126d152969': 'bottom_right_corner',
    '24e124128c067999': 'indoor',
    '24e124785c389818': 'bottom_left_corner',
    '24e124785c421885': 'top_right_corner'
}

# 한국 시간대 설정
korea_tz = pytz.timezone('Asia/Seoul')

# 실행 시점의 날짜를 기준으로 전날의 날짜를 계산
today_kst = datetime.now(korea_tz)
yesterday_kst = today_kst - timedelta(days=1)

# 전날의 시작과 끝 시간을 한국 시간대로 설정
start_time_kst = korea_tz.localize(datetime(yesterday_kst.year, yesterday_kst.month, yesterday_kst.day, 0, 0, 0))
end_time_kst = start_time_kst + timedelta(days=1)

# UTC로 시간 변환
start_time_utc = start_time_kst.astimezone(pytz.utc)
end_time_utc = end_time_kst.astimezone(pytz.utc)

In [5]:
def create_client(url, token, org):
    """
    Influx DB 연결 Client를 생성합니다.
    
    :param url: InfluxDB 연결 주소
    :param token: InfluxDB 토큰
    :param org:  InfluxDB 조직
    :return: InfluxDBClient
    """
    return InfluxDBClient(url=url, token=token, org=org)

def query_to_dataframe(client, query, field = "location"):
    """
    구성된 쿼리를 실행하고 전달받은 데이터를 Dataframe으로 만듭니다.
    
    :param field: 기본값 location
    :param client: InfluxDBClient
    :param query: 요청할 쿼리
    :return: DataFrame 
    """
    result = client.query_api().query(query=query)
    results = []
    
    for table in result:
        for record in table.records:
            results.append({
                "time": record.get_time(),
                "value": record.get_value(),
                "place": record.values.get("place"),
                "location": record.values.get(field),
                "device": record.values.get("device")
            })
    
    df = pd.DataFrame(results)
    df['time'] = df['time'].astype(str).str.replace(r'\+00:00$', '', regex=True)
    return df

def save_csv(df, file_pattern, directory):
    
    """
    DataFrame을 CSV로 변환하여 저장합니다.
    
    :param df: DataFrame
    :param file_pattern: 파일 이름 패턴
    :param directory: 저장할 위치
    """
    # 파일 경로를 확인 하고 없다면 생성 합니다.
    if not os.path.exists(directory):
        os.makedirs(directory)
        
    previous_date = datetime.now() - timedelta(days=1)
    filename = f"{directory}{previous_date.strftime(file_pattern)}"
    df.to_csv(filename, index=False)
        
def update_location(df):
    """
    환경 센서 Data에서 Device ID를 확인 하여 'location' 열을 업데이트 합니다.
    
    :param df: 환경 센서 DataFrame
    :return: 'location' 열을 업데이트한 DataFrame
    """
    df['location'] = df['device'].map(location_mapping)
    return df

def merge_data(directory_path):
    """
    CSV 파일들을 하나의 DataFrame으로 병합하고 정렬 합니다.
    
    :param directory_path: 파일 경로 
    :return: 병합된 DataFrame
    """
    # 파일 내의 모든 CSV 파일 목록을 생성
    csv_files = [file for file in os.listdir(directory_path) if file.endswith('.csv')]

    # 모든 CSV 파일을 DataFrame으로 읽어와 하나로 병합
    data_frames = []
    for file in csv_files:
        file_path = os.path.join(directory_path, file)
        df = pd.read_csv(file_path)
        df['time'] = pd.to_datetime(df['time'])
        data_frames.append(df)

    merged_df = pd.concat(data_frames, ignore_index=True)

    # 해당 컬럼들로 재구성
    columns_to_keep = ['time', 'value', 'place', 'location']
    filtered_df = merged_df[columns_to_keep]

    # 'place', 'time' 으로 정렬
    sorted_df = filtered_df.sort_values(['place', 'time'])

    return sorted_df

def df_hourly_calculate(dataframe, change_column_name, calculate):
    """
    DataFrame에서 value를 시간별로 calculate 함수에 따라 계산합니다.
    계산 후 value 컬럼을 지정한 이름으로 변경합니다.
        
    :param dataframe: 계산할 DataFrame
    :param change_column_name: 계산 후 value 컬럼을 지정한 이름으로 변경
    :param calculate: 문자열 형태로 계산할 함수명 ('sum', 'mean', 'diff' 등)
    :return: 시간별 계산된 DataFrame
    """
    # 시간 컬럼이 datetime형태인지 확인하고 설정후 Index로 설정
    if not pd.api.types.is_datetime64_any_dtype(dataframe.index):
        dataframe['time'] = pd.to_datetime(dataframe['time'])
        dataframe.set_index('time', inplace=True)

    # 시간별로 그룹화하여 마지막 값을 선택 (누적값의 경우)
    if calculate == 'diff':
        # 누적값의 마지막 값을 시간별로 선택하고 차분 계산
        result = dataframe['value'].resample('H').last().diff().rename(change_column_name)
    else:
        # 기타 계산
        result = dataframe.resample('H').agg({'value': calculate}).rename(columns={'value': change_column_name})

    # Null 값 채우기 (forward fill)
    result.fillna(method='bfill', inplace=True)

    return result

def merge_dataframes(df_list):
    """
    여러 DataFrame을 병합합니다. 
    
    :param df_list: 병합할 DataFrame 리스트
    :return: 병합된 DataFrame
    """
    merged_df = df_list[0]
    for df in df_list[1:]:
        merged_df = pd.merge(merged_df, df, left_index=True, right_index=True, how='outer', validate=None)
    return merged_df

def calculate_attendance(dataframe):
    # time 컬럼을 datetime 타입으로 변환
    dataframe['time'] = pd.to_datetime(dataframe['time'])

    # 시간대별로 그룹화 (시간, 분, 초까지 모두 고려)
    grouped = dataframe.groupby(['time', 'location'])['value'].sum().unstack().fillna(0)

    # 입장과 퇴장 데이터 분리
    grouped['in'] = grouped.filter(like='_in').sum(axis=1)
    grouped['out'] = grouped.filter(like='_out').sum(axis=1)

    # 순 입장 인원 계산 (재실 인원)
    grouped['net'] = grouped['in'] - grouped['out']

    # 누적 재실 인원 계산
    cumulative = 0  # 초기값 설정
    cumulative_results = []  # 누적 결과를 저장할 리스트
    for net in grouped['net']:
        cumulative += net
        if cumulative < 0:
            cumulative = 0  # 음수는 0으로 처리
        cumulative_results.append(cumulative)

    grouped['cumulative'] = cumulative_results

    return grouped[['in', 'out', 'cumulative']]

In [14]:
# DB 연결 및 쿼리 실행
client = create_client(url, token, org)

# 전력(W) 조회(class a : main) Flux 쿼리
query_power_total_data = f'''
import "experimental"
from(bucket: "powermetrics_data")
  |> range(start: {start_time_utc.strftime('%Y-%m-%dT%H:%M:%SZ')}, stop: {end_time_utc.strftime('%Y-%m-%dT%H:%M:%SZ')})
  |> filter(fn: (r) => r["description"] == "w")
  |> filter(fn: (r) => r["place"] == "class_a")
  |> filter(fn: (r) => r["phase"] == "total")
  |> filter(fn: (r) => r["location"] == "main")
  |> aggregateWindow(every: 2m, fn: mean, createEmpty: false)
  |> map(fn: (r) => ({{r with _time: experimental.addDuration(d: 9h, to: r._time)}}))
  |> keep(columns: ["_time", "_value", "place", "location", "device"])
'''

# 전력 사용량 조회(class a : main) Flux 쿼리
query_power_usage_total_data = f'''
import "experimental"
from(bucket: "powermetrics_data")
  |> range(start: {start_time_utc.strftime('%Y-%m-%dT%H:%M:%SZ')}, stop: {end_time_utc.strftime('%Y-%m-%dT%H:%M:%SZ')})
  |> filter(fn: (r) => r["phase"] == "kwh")
  |> filter(fn: (r) => r["place"] == "class_a")
  |> filter(fn: (r) => r["description"] == "sum")
  |> filter(fn: (r) => r["location"] == "main")
  |> aggregateWindow(every: 2m, fn: mean, createEmpty: false)
  |> map(fn: (r) => ({{r with _time: experimental.addDuration(d: 9h, to: r._time)}}))
  |> keep(columns: ["_time", "_value", "place", "location", "device"])
'''

# 온도 조회 Flux 쿼리
query_temperature_data = f'''
import "experimental"
from(bucket: "environmentalsensors_data")
  |> range(start: {start_time_utc.strftime('%Y-%m-%dT%H:%M:%SZ')}, stop: {end_time_utc.strftime('%Y-%m-%dT%H:%M:%SZ')})
  |> filter(fn: (r) => r["measurement"] == "temperature")
  |> filter(fn: (r) => r["place"] == "class_a")
  |> aggregateWindow(every: 2m, fn: mean, createEmpty: false)
  |> map(fn: (r) => ({{r with _time: experimental.addDuration(d: 9h, to: r._time)}}))
  |> keep(columns: ["_time", "_value", "place", "location", "device"])
'''

# 이산화탄소 조회 Flux 쿼리
query_co2_data = f'''
import "experimental"
from(bucket: "environmentalsensors_data")
  |> range(start: {start_time_utc.strftime('%Y-%m-%dT%H:%M:%SZ')}, stop: {end_time_utc.strftime('%Y-%m-%dT%H:%M:%SZ')})
  |> filter(fn: (r) => r["place"] == "class_a")
  |> filter(fn: (r) => r["measurement"] == "co2")
  |> aggregateWindow(every: 2m, fn: mean, createEmpty: false)
  |> map(fn: (r) => ({{r with _time: experimental.addDuration(d: 9h, to: r._time)}}))
  |> keep(columns: ["_time", "_value", "place", "location", "device"])
'''

# 조도 조회 Flux 쿼리
query_illumination_data = f'''
import "experimental"
from(bucket: "environmentalsensors_data")
  |> range(start: {start_time_utc.strftime('%Y-%m-%dT%H:%M:%SZ')}, stop: {end_time_utc.strftime('%Y-%m-%dT%H:%M:%SZ')})
  |> filter(fn: (r) => r["place"] == "class_a")
  |> filter(fn: (r) => r["measurement"] == "illumination")
  |> aggregateWindow(every: 2m, fn: mean, createEmpty: false)
  |> map(fn: (r) => ({{r with _time: experimental.addDuration(d: 9h, to: r._time)}}))
  |> keep(columns: ["_time", "_value", "place", "location", "device"])
'''

# 습도 조회 Flux 쿼리
query_humidity_data = f'''
import "experimental"
from(bucket: "environmentalsensors_data")
  |> range(start: {start_time_utc.strftime('%Y-%m-%dT%H:%M:%SZ')}, stop: {end_time_utc.strftime('%Y-%m-%dT%H:%M:%SZ')})
  |> filter(fn: (r) => r["place"] == "class_a")
  |> filter(fn: (r) => r["measurement"] == "humidity")
  |> aggregateWindow(every: 2m, fn: mean, createEmpty: false)
  |> map(fn: (r) => ({{r with _time: experimental.addDuration(d: 9h, to: r._time)}}))
  |> keep(columns: ["_time", "_value", "place", "location", "device"])
'''

# 이동 감지 카운터 조회 Flux 쿼리
query_counter_data = f'''
import "experimental"
from(bucket: "milesight")
  |> range(start: {start_time_utc.strftime('%Y-%m-%dT%H:%M:%SZ')}, stop: {end_time_utc.strftime('%Y-%m-%dT%H:%M:%SZ')})
  |> filter(fn: (r) => r["place"] == "class_a")
  |> filter(fn: (r) => r["_field"] == "line_periodic_data_1_out" or r["_field"] == "line_periodic_data_1_in")
  |> aggregateWindow(every: 2m, fn: mean, createEmpty: false)
  |> map(fn: (r) => ({{r with _time: experimental.addDuration(d: 9h, to: r._time)}}))
  |> keep(columns: ["_time", "_value", "place", "_field", "device"])
'''

print(query_counter_data)

# 전력 CSV 생성
df_power = query_to_dataframe(client, query_power_total_data)
print(df_power.head(2))
save_csv(df_power, "%m_%d_power_total_data.csv", "all_data/power/total/")

# 전력 사용량 CSV 생성
df_power_usage = query_to_dataframe(client, query_power_usage_total_data)
save_csv(df_power_usage, "%m_%d_power_usage_total_data.csv", "all_data/power_usage/total/")
print(df_power_usage.head(2))

# 온도 CSV 생성
df_temperature = query_to_dataframe(client, query_temperature_data)
df_temperature_fix = update_location(df_temperature)
print(df_temperature_fix.head(2))
save_csv(df_temperature_fix, "%m_%d_temperature_data.csv", "all_data/temperature/total/")

# co2 CSV 생성
df_co2 = query_to_dataframe(client, query_co2_data)
df_co2_fix = update_location(df_co2)
print(df_co2_fix.head(2))
save_csv(df_co2_fix, "%m_%d_co2_data.csv", "all_data/co2/")

# 조도 CSV 생성
df_illumination = query_to_dataframe(client, query_illumination_data)
update_location(df_illumination)
print(df_illumination.head(2))
save_csv(df_illumination, "%m_%d_illumination_data.csv", "all_data/illumination/")

# 습도 CSV 생성
df_humidity = query_to_dataframe(client, query_humidity_data)
update_location(df_humidity)
print(df_humidity.head(2))
save_csv(df_humidity, "%m_%d_humidity_data.csv", "all_data/humidity/")

# 이동 감지 카운터 CSV 생성
df_counter = query_to_dataframe(client, query_counter_data, "_field")
print(df_counter.head(2))
save_csv(df_counter, "%m_%d_counter_data.csv", "all_data/counter/total/")

# 연결 Client 종료
client.close()


import "experimental"
from(bucket: "milesight")
  |> range(start: 2024-04-30T15:00:00Z, stop: 2024-05-01T15:00:00Z)
  |> filter(fn: (r) => r["place"] == "class_a")
  |> filter(fn: (r) => r["_field"] == "line_periodic_data_1_out" or r["_field"] == "line_periodic_data_1_in")
  |> aggregateWindow(every: 2m, fn: mean, createEmpty: false)
  |> map(fn: (r) => ({r with _time: experimental.addDuration(d: 9h, to: r._time)}))
  |> keep(columns: ["_time", "_value", "place", "_field", "device"])

                  time   value    place location     device
0  2024-05-01 00:02:00  5012.5  class_a     main  gems-3500
1  2024-05-01 00:04:00  4723.0  class_a     main  gems-3500
                  time   value    place location     device
0  2024-05-01 00:02:00  759.45  class_a     main  gems-3500
1  2024-05-01 00:04:00  759.65  class_a     main  gems-3500
                  time  value    place             location            device
0  2024-05-01 00:02:00   23.1  class_a  bottom_right_corner  24e124126d

In [8]:
# 내가 작성한 곳 

# DB 연결 및 쿼리 실행
client = create_client(url, token, org)

# 전력(W) 조회(class a : main) Flux 쿼리
query_power_device_data = f'''
import "experimental"
from(bucket: "powermetrics_data")
  |> range(start: {start_time_utc.strftime('%Y-%m-%dT%H:%M:%SZ')}, stop: {end_time_utc.strftime('%Y-%m-%dT%H:%M:%SZ')})
  |> filter(fn: (r) => r["phase"] == "total")
  |> filter(fn: (r) => r["description"] == "w")
  |> filter(fn: (r) => r["place"] == "class_a")
  |> filter(fn: (r) => r["location"] != "main" and r["location"] != "outdoor_unit_room_light")
  |> aggregateWindow(every: 2m, fn: mean, createEmpty: false)
  |> map(fn: (r) => ({{r with _time: experimental.addDuration(d: 9h, to: r._time)}}))
  |> keep(columns: ["_time", "_value", "place", "location", "device"])
'''

# 전력 CSV 생성
df_power = query_to_dataframe(client, query_power_device_data)
print(df_power.head(2))
save_csv(df_power, "%m_%d_power_device_data.csv", "all_data/power/device/")

# 전력 사용량 조회(class a : main) Flux 쿼리
query_power_usage_device_data = f'''
import "experimental"
from(bucket: "powermetrics_data")
  |> range(start: {start_time_utc.strftime('%Y-%m-%dT%H:%M:%SZ')}, stop: {end_time_utc.strftime('%Y-%m-%dT%H:%M:%SZ')})
  |> filter(fn: (r) => r["phase"] == "kwh")
  |> filter(fn: (r) => r["place"] == "class_a")
  |> filter(fn: (r) => r["description"] == "sum")
  |> filter(fn: (r) => r["location"] != "main" and r["location"] != "outdoor_unit_room_light")
  |> aggregateWindow(every: 2m, fn: mean, createEmpty: false)
  |> map(fn: (r) => ({{r with _time: experimental.addDuration(d: 9h, to: r._time)}}))
  |> keep(columns: ["_time", "_value", "place", "location", "device"])
'''

# 전력 사용량 CSV 생성
df_power_usage = query_to_dataframe(client, query_power_usage_device_data)
save_csv(df_power_usage, "%m_%d_power_usage_device_data.csv", "all_data/power_usage/device/")
print(df_power_usage.head(2))

# 연결 Client 종료
client.close()

                  time  value    place        location     device
0  2024-05-01 00:02:00  200.0  class_a  ac_indoor_unit  gems-3500
1  2024-05-01 00:04:00  199.0  class_a  ac_indoor_unit  gems-3500
                  time  value    place        location     device
0  2024-05-01 00:02:00   39.8  class_a  ac_indoor_unit  gems-3500
1  2024-05-01 00:04:00   39.8  class_a  ac_indoor_unit  gems-3500


In [68]:
power_path = 'all_data/power/total'
power_total_df = merge_data(power_path)
print(power_total_df.head(2), "\n")

power_usage_path = 'all_data/power_usage/total'
power_usage_total_df = merge_data(power_usage_path)
print(power_usage_total_df.head(2), "\n")

temperature_path = 'all_data/temperature'
temperature_total_df = merge_data(temperature_path)
print(temperature_total_df.head(2), "\n")

co2_path = 'all_data/co2'
co2_total_df = merge_data(co2_path)
print(co2_total_df.head(2), "\n")

illumination_path = 'all_data/illumination'
illumination_total_df = merge_data(illumination_path)
print(illumination_total_df.head(2), "\n")

humidity_path = 'all_data/humidity'
humidity_total_df = merge_data(humidity_path)
print(humidity_total_df.head(2), "\n")

counter_path = 'all_data/counter/total'
counter_total_df = merge_data(counter_path)
print(counter_total_df.head(2), "\n")

                    time  value    place location
9345 2024-04-15 00:02:00  152.0  class_a     main
9346 2024-04-15 00:04:00  150.5  class_a     main 

                    time  value    place location
5743 2024-04-15 00:02:00  264.7  class_a     main
5744 2024-04-15 00:04:00  264.7  class_a     main 

                     time  value    place             location
24282 2024-04-15 00:02:00   24.0  class_a  bottom_right_corner
24986 2024-04-15 00:02:00   24.6  class_a               indoor 

                     time  value    place location
10059 2024-04-15 00:02:00  658.0  class_a   indoor
10060 2024-04-15 00:04:00  659.0  class_a   indoor 

                    time  value    place             location
8618 2024-04-15 00:02:00   49.5  class_a  bottom_right_corner
9322 2024-04-15 00:02:00   45.5  class_a               indoor 

                     time  value    place             location
21416 2024-04-15 00:02:00   49.5  class_a  bottom_right_corner
22120 2024-04-15 00:02:00   45.5  cl

In [69]:
temperature_total_df = df_hourly_calculate(temperature_total_df, "average_temperature(°C)", "mean")
humidity_total_df = df_hourly_calculate(humidity_total_df, "average_humidity(%)", "mean")
co2_total_df = df_hourly_calculate(co2_total_df, "average_co2(ppm)", "mean")
illumination_total_df = df_hourly_calculate(illumination_total_df, "average_illumination(lux)", "mean")
counter_total_df = calculate_attendance(counter_total_df)

In [70]:
# 데이터프레임 병합
total_environment_data_df = merge_dataframes([counter_total_df, temperature_total_df, humidity_total_df, co2_total_df, illumination_total_df])

# 결과 확인
print(total_environment_data_df)

# 인덱스를 일반 컬럼으로 변경
final_df1 = total_environment_data_df.reset_index()

# CSV 파일로 저장
final_df1.to_csv('all_data/class_a_environmental_sensor.csv', index=False)

                       in   out  cumulative  average_temperature(°C)  \
time                                                                   
2024-04-15 00:00:00   NaN   NaN         NaN                23.918247   
2024-04-15 01:00:00   NaN   NaN         NaN                23.331073   
2024-04-15 02:00:00   NaN   NaN         NaN                23.240556   
2024-04-15 03:00:00   NaN   NaN         NaN                23.148472   
2024-04-15 04:00:00   NaN   NaN         NaN                22.955322   
...                   ...   ...         ...                      ...   
2024-05-01 20:00:00  11.0  11.0         0.0                23.611667   
2024-05-01 21:00:00  15.0   9.0         6.0                23.312037   
2024-05-01 22:00:00   9.0  10.0         5.0                23.214962   
2024-05-01 23:00:00   0.0   0.0         5.0                23.116479   
2024-05-02 00:00:00   0.0   0.0         5.0                23.016667   

                     average_humidity(%)  average_co2(ppm)  \
t

In [9]:
# 전체 전력/전력량
power_total_df = df_hourly_calculate(power_total_df, "total_power(Wh)", "sum")
power_usage_total_df = df_hourly_calculate(power_usage_total_df, 'total_power_usage(Kwh)', 'diff')

# 실내기 전력/전력량

# 실외기 전력/전력량

# 자동문 전력/전력량


In [10]:
total_power_data_df = merge_dataframes([power_total_df, power_usage_total_df])
print(total_power_data_df)
final_df2 = total_power_data_df.reset_index()
final_df2.to_csv('all_data/class_a_power_sensor.csv', index=False)

                     total_power(Wh)  total_power_usage(Kwh)
time                                                        
2024-04-15 00:00:00           2234.5                    0.20
2024-04-15 01:00:00           6152.5                    0.20
2024-04-15 02:00:00           5315.5                    0.20
2024-04-15 03:00:00           3457.5                    0.10
2024-04-15 04:00:00           6012.5                    0.20
...                              ...                     ...
2024-05-01 20:00:00          31287.5                    1.00
2024-05-01 21:00:00          17475.5                    0.50
2024-05-01 22:00:00          16100.5                    0.50
2024-05-01 23:00:00          17601.5                    0.50
2024-05-02 00:00:00            961.0                    0.05

[409 rows x 2 columns]


In [17]:
# 환경 설정
import pandas as pd
import os

# CSV 파일을 로드하고 데이터프레임을 생성
def load_and_process_data(directory, identifier):
    frames = []
    print(f"디렉토리 검색: {directory}")  # 디렉토리 출력
    for file_name in os.listdir(directory):
        print(f"검사 중인 파일: {file_name}")  # 검사 중인 파일 이름 출력
        if identifier in file_name:
            print(f"로드 중인 파일: {file_name}")  # 로드하는 파일 이름 출력
            df = pd.read_csv(os.path.join(directory, file_name))
            print(f"로드된 데이터:\n{df.head()}")  # 로드된 데이터의 첫 5행 출력
            df['time'] = pd.to_datetime(df['time'])
            frames.append(df)
    if frames:
        combined_df = pd.concat(frames, ignore_index=True)
        print(f"병합된 데이터프레임 정보:\n{combined_df.info()}")  # 병합 후 데이터프레임 정보 출력
        return combined_df
    else:
        print("데이터가 없습니다.")  # 데이터가 없는 경우 메시지 출력
        return pd.DataFrame()
    

# 각 장치 유형별 데이터 로드
base_directory = 'all_data/power/device'
ac_in_df = load_and_process_data(base_directory, 'ac_in')
ac_out_df = load_and_process_data(base_directory, 'ac_out')
auto_indoor_df = load_and_process_data(base_directory, 'auto_indoor')

# 데이터 저장
def save_data(df, filename):
    directory = os.path.dirname(filename)
    if not os.path.exists(directory):
        os.makedirs(directory)
    df.to_csv(filename, index=False)
    print(f"데이터가 저장되었습니다: {filename}")

# 데이터 프레임 내용 출력
print("AC Indoor DataFrame (a/c_in_df):")
print(ac_in_df.head())  # 첫 5개 행 출력
print("\nDataFrame Info:")
print(ac_in_df.info())  # 데이터 프레임 정보 출력

print("\nAC Outdoor DataFrame (a/c_in_df):")
print(ac_out_df.head())
print("\nDataFrame Info:")
print(ac_out_df.info())

print("\nAuto Indoor DataFrame (auto_indoor_unit):")
print(auto_indoor_df.head())
print("\nDataFrame Info:")
print(auto_indoor_df.info())

# 데이터 저장
print("데이터 저장 중..")
save_data(ac_in_df, 'path/to/save/ac_in.csv')
save_data(ac_out_df, 'path/to/save/ac_out.csv')
save_data(auto_indoor_df, 'path/to/save/auto_indoor.csv')

디렉토리 검색: all_data/power/device
검사 중인 파일: 04_15_power_device_data.csv
검사 중인 파일: 04_16_power_device_data.csv
검사 중인 파일: 04_17_power_device_data.csv
검사 중인 파일: 04_18_power_device_data.csv
검사 중인 파일: 04_19_power_device_data.csv
검사 중인 파일: 04_20_power_device_data.csv
검사 중인 파일: 04_21_power_device_data.csv
검사 중인 파일: 04_22_power_device_data.csv
검사 중인 파일: 04_23_power_device_data.csv
검사 중인 파일: 04_24_power_device_data.csv
검사 중인 파일: 04_25_power_device_data.csv
검사 중인 파일: 04_26_power_device_data.csv
검사 중인 파일: 04_27_power_device_data.csv
검사 중인 파일: 04_28_power_device_data.csv
검사 중인 파일: 04_29_power_device_data.csv
검사 중인 파일: 04_30_power_device_data.csv
검사 중인 파일: 05_01_power_device_data.csv
데이터가 없습니다.
디렉토리 검색: all_data/power/device
검사 중인 파일: 04_15_power_device_data.csv
검사 중인 파일: 04_16_power_device_data.csv
검사 중인 파일: 04_17_power_device_data.csv
검사 중인 파일: 04_18_power_device_data.csv
검사 중인 파일: 04_19_power_device_data.csv
검사 중인 파일: 04_20_power_device_data.csv
검사 중인 파일: 04_21_power_device_data.csv
검사 중인 파일: 04_22

In [13]:
import pandas as pd
import os

def load_data(directory, identifier):
    combined_df = pd.DataFrame()  # 초기 빈 데이터프레임 생성
    for file_name in os.listdir(directory):
        if identifier in file_name:  # 식별자를 포함하는 파일 이름 확인
            file_path = os.path.join(directory, file_name)
            df = pd.read_csv(file_path)
            combined_df = pd.concat([combined_df, df], ignore_index=True)
    return combined_df

# 기본 디렉토리 설정
base_directory = 'all_data/power/device'  # 경로는 실제 환경에 맞게 조정해주세요.

# 각 장치 유형별 데이터 로드
a_c_in_df = load_data(base_directory, 'ac_in')
a_c_out_df = load_data(base_directory, 'ac_out')
auto_indoor_df = load_data(base_directory, 'auto_indoor')

print("A/C Indoor Unit Data:")
print(a_c_in_df.head())

print("\nA/C Outdoor Unit Data:")
print(a_c_out_df.head())

print("\nAuto Indoor Unit Data:")
print(auto_indoor_df.head())


A/C Indoor Unit Data:
Empty DataFrame
Columns: []
Index: []

A/C Outdoor Unit Data:
Empty DataFrame
Columns: []
Index: []

Auto Indoor Unit Data:
Empty DataFrame
Columns: []
Index: []


In [20]:
import pandas as pd

# csv 파일에서 데이터 로드 하여 데이터프레임으로 반환
def load_data(filepath):
    return pd.read_csv(filepath)

# 전체 데이터셋 로드
power_data_path = '/path/to/04_16_power_device_data.csv'  # 이 경로를 업데이트하세요
power_data = load_data(power_data_path)

# 각 장치 유형별로 데이터를 필터링하여 데이터프레임 생성
ac_in_df = power_data[power_data['location'].str.contains('ac_in', case=False, na=False)]
ac_out_df = power_data[power_data['location'].str.contains('ac_out', case=False, na=False)]
auto_indoor_df = power_data[power_data['location'].str.contains('auto_indoor', case=False, na=False)]

# 각 데이터프레임의 첫 몇 줄을 출력하여 확인
print("AC 인도어 데이터프레임:")
print(ac_in_df.head(), "\n")

print("AC 아웃도어 데이터프레임:")
print(ac_out_df.head(), "\n")

print("자동 인도어 데이터프레임:")
print(auto_indoor_df.head(), "\n")

FileNotFoundError: [Errno 2] No such file or directory: '/path/to/04_16_power_device_data.csv'